In [1]:
import tapas.datasets
import tapas.generators
import tapas.threat_models
import tapas.attacks
import tapas.report

In [2]:
data = tapas.datasets.TabularDataset.read("data/microdata-2011")

In [3]:
data.data.head()

,Region,Residence Type,Family Composition,Population Base,Sex,Age,Marital Status,Student,Country of Birth,Health,Ethnic Group,Religion,Economic Activity,Occupation,Industry,Hours worked per week,Approximated Social Grade
0,E12000001,H,2,1,2,6,2,2,1,2,1,2,5,8,2,-9,4
1,E12000001,H,5,1,1,4,1,2,1,1,1,2,1,8,6,4,3
2,E12000001,H,3,1,2,4,1,2,1,1,1,1,1,6,11,3,4
3,E12000001,H,3,1,2,2,1,2,1,2,1,2,1,7,7,3,2
4,E12000001,H,3,1,1,5,4,2,1,1,1,2,1,1,4,3,2


Notes
- there is no continuous variable in this census data set. does this matter for some generators?

In [4]:
display(type(data))
dir(data)
type(data.data)
data.data.head()


tapas.datasets.dataset.TabularDataset

,Region,Residence Type,Family Composition,Population Base,Sex,Age,Marital Status,Student,Country of Birth,Health,Ethnic Group,Religion,Economic Activity,Occupation,Industry,Hours worked per week,Approximated Social Grade
0,E12000001,H,2,1,2,6,2,2,1,2,1,2,5,8,2,-9,4
1,E12000001,H,5,1,1,4,1,2,1,1,1,2,1,8,6,4,3
2,E12000001,H,3,1,2,4,1,2,1,1,1,1,1,6,11,3,4
3,E12000001,H,3,1,2,2,1,2,1,2,1,2,1,7,7,3,2
4,E12000001,H,3,1,1,5,4,2,1,1,1,2,1,1,4,3,2


the first 2 rows are odd??

In [5]:
generator = tapas.generators.Raw()

it's not clear to me how the generator is used downstream. ie, in this quickstart example, we only define the generator, and then pass it on to the threat model (through `sdg_knowledge`). 
- this must imply that generator.train() is called somewhere in the attack?
    - The combination of all inputs to threat_model make all parts work together 
    - Thus, it is crucial to define the data type properly when using our own data/generator (following the instructions above)

Below we see how to use `generator.generate()`.

In [6]:
generator.fit(data.data)
generator.generate(3)

,Region,Residence Type,Family Composition,Population Base,Sex,Age,Marital Status,Student,Country of Birth,Health,Ethnic Group,Religion,Economic Activity,Occupation,Industry,Hours worked per week,Approximated Social Grade
342829,E12000002,H,5,1,2,2,2,2,2,1,3,6,9,-9,-9,-9,4
565248,E12000002,C,-9,1,1,6,1,2,1,5,1,2,8,-9,-9,-9,-9
317699,E12000005,H,2,1,1,6,2,2,2,2,5,7,2,1,4,4,2


In [7]:
dir(generator)
generator.generate??
tapas.generators.Raw??

Init signature: tapas.generators.Raw()
Source:        
class Raw(Generator):
    """This generator simply samples from the real data."""
    def __init__(self):
        super().__init__()

    def fit(self, dataset):
        self.dataset = dataset
        self.trained = True

    def generate(self, num_samples = None, random_state = None):
        if self.trained: 
            if num_samples is None:
                return self.dataset
            return self.dataset.sample(num_samples, random_state = random_state)
        else:
            raise RuntimeError("No dataset provided to generator")
        
    def __call__(self, dataset, num_samples, random_state = None):
        self.fit(dataset)
        return self.generate(num_samples, random_state = random_state)

    @property
    def label(self):
        return "Raw"
File:           ~/repositories/projects/GANS/explore-attacks/.venv/lib/python3.9/site-packages/tapas/generators/generator.py
Type:           ABCMeta
Subclasses:     

Define attacker's knowledge about the data set 

In [8]:
data_knowledge = tapas.threat_models.AuxiliaryDataKnowledge(
      data,
      auxiliary_split=0.5,
      num_training_records=5000
)

Define knowledge about the generator

In [9]:
sdg_knowledge = tapas.threat_models.BlackBoxKnowledge(
    generator,
    num_synthetic_records=5000,
)

Define goal of the attacker: membership inference attack

In [70]:
target_record_ids = [0, 1, 2, 3]

In [71]:
threat_model = tapas.threat_models.TargetedMIA(
   attacker_knowledge_data=data_knowledge,
   target_record=data.get_records(target_record_ids),
   attacker_knowledge_generator=sdg_knowledge,
   generate_pairs=True, # see API. SOURCE?
   replace_target=True
)

should I not get multiple records here?

In [72]:
threat_model.target_record.data

,Region,Residence Type,Family Composition,Population Base,Sex,Age,Marital Status,Student,Country of Birth,Health,Ethnic Group,Religion,Economic Activity,Occupation,Industry,Hours worked per week,Approximated Social Grade
0,E12000001,H,2,1,2,6,2,2,1,2,1,2,5,8,2,-9,4


In [80]:
threat_model._target_records # so here we have all the records
for r in threat_model._target_records:
    # display(r.data)
    pass 

# can we somewhere in the attacks specify which records we want to use?

In [81]:
data.data.iloc[target_record_ids]

,Region,Residence Type,Family Composition,Population Base,Sex,Age,Marital Status,Student,Country of Birth,Health,Ethnic Group,Religion,Economic Activity,Occupation,Industry,Hours worked per week,Approximated Social Grade
0,E12000001,H,2,1,2,6,2,2,1,2,1,2,5,8,2,-9,4
1,E12000001,H,5,1,1,4,1,2,1,1,1,2,1,8,6,4,3
2,E12000001,H,3,1,2,4,1,2,1,1,1,1,1,6,11,3,4
3,E12000001,H,3,1,2,2,1,2,1,2,1,2,1,7,7,3,2


In [67]:
data.get_records(target_record_ids).data

,Region,Residence Type,Family Composition,Population Base,Sex,Age,Marital Status,Student,Country of Birth,Health,Ethnic Group,Religion,Economic Activity,Occupation,Industry,Hours worked per week,Approximated Social Grade
0,E12000001,H,2,1,2,6,2,2,1,2,1,2,5,8,2,-9,4
1,E12000001,H,5,1,1,4,1,2,1,1,1,2,1,8,6,4,3
2,E12000001,H,3,1,2,4,1,2,1,1,1,1,1,6,11,3,4
3,E12000001,H,3,1,2,2,1,2,1,2,1,2,1,7,7,3,2


#### Attack

In [31]:
attacker = tapas.attacks.GroundhogAttack() # I suppose the relevant features of the attack are defined in the class?

This is equivalent to a particular instantiation of a larger class of attacks. The current code on the website does not work though.

In [32]:
attacker.train(threat_model, num_samples=100)

In [33]:
attack_summary = threat_model.test(attacker, num_samples=100)

In [84]:
metrics = attack_summary.get_metrics()


In [85]:
metrics

,dataset,target_id,generator,attack,accuracy,true_positive_rate,false_positive_rate,mia_advantage,privacy_gain,auc,effective_epsilon
0,data/microdata-2011 (AUX),0,Raw,Groundhog,0.49,0.52,0.54,-0.02,1.02,0.515,0.251314


In [86]:
attack_summary.tp / attack_summary.fp

0.9629629629629629

#### Attack multiple records

- I don't think this works because of the way the attacker knowledge on the dataset is modeled 

In [36]:
def attack_record(id):
    threat_model = tapas.threat_models.TargetedMIA(
        attacker_knowledge_data=data_knowledge,
        target_record=data.get_records(id),
        attacker_knowledge_generator=sdg_knowledge,
        generate_pairs=True, # see API. SOURCE?
        replace_target=True
    )   
    attacker = tapas.attacks.GroundhogAttack()
    attacker.train(threat_model, num_samples=100)
    attack_summary = threat_model.test(attacker, num_samples=100)
    metrics = attack_summary.get_metrics()
    return metrics 

In [37]:
metrics = []
for i in target_record_ids:
    print(i)
    m = attack_record(i)
    metrics.append(m)

[                     dataset target_id generator     attack  accuracy  \
 0  data/microdata-2011 (AUX)         0       Raw  Groundhog      0.49   
 
    true_positive_rate  false_positive_rate  mia_advantage  privacy_gain  \
 0                0.44                 0.46          -0.02          1.02   
 
       auc  effective_epsilon  
 0  0.5082           0.405465  ,
                      dataset target_id generator     attack  accuracy  \
 0  data/microdata-2011 (AUX)         0       Raw  Groundhog      0.49   
 
    true_positive_rate  false_positive_rate  mia_advantage  privacy_gain  \
 0                0.48                  0.5          -0.02          1.02   
 
       auc  effective_epsilon  
 0  0.5304           0.287682  ,
                      dataset target_id generator     attack  accuracy  \
 0  data/microdata-2011 (AUX)         0       Raw  Groundhog      0.52   
 
    true_positive_rate  false_positive_rate  mia_advantage  privacy_gain  \
 0                0.44              

In [43]:
import pandas as pd 
pd.concat(metrics)

,dataset,target_id,generator,attack,accuracy,true_positive_rate,false_positive_rate,mia_advantage,privacy_gain,auc,effective_epsilon
0,data/microdata-2011 (AUX),0,Raw,Groundhog,0.49,0.44,0.46,-0.02,1.02,0.5082,0.405465
0,data/microdata-2011 (AUX),0,Raw,Groundhog,0.49,0.48,0.50,-0.02,1.02,0.5304,0.287682
0,data/microdata-2011 (AUX),0,Raw,Groundhog,0.52,0.44,0.40,0.04,0.96,0.5426,0.619039


Questions
- do I have to run the attack for each target record separately? -- see comments above. there must be away to run attacks automatically on several records 
- effective epsilon calculation?

What do we learn from this?